In [3]:
import matplotlib.pyplot as plt
import cv2
from functions import *
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
labels = [
    ['1_real'],
    ['50_cents'],
    ['25_cents'],
    ['5_cents'],
    ['10_cents'],
    ['50_cents', '25_cents', '1_real', '5_cents', '10_cents'],
    ['50_cents', '25_cents', '10_cents'],
    ['1_real', '50_cents'],
    ['1_real', '50_cents', '10_cents'],
    ['25_cents', '10_cents']
]

training_data = list()

for i in range(10):
    img_gray, img_rgb = prepare_img(f'images/img{i}.png')
    circles = find_circles(img_gray)
    circles = treat_circles(circles)
    
    if circles is not None:
        
        filtered_circles = filter_inner_circles(circles, 10)
        coins_histograms = create_histogram_of_visual_words(img_rgb, filtered_circles)
        update_training_data(coins_histograms, labels[i], training_data)
    
    
save_training_data(training_data, 'training_data.pkl')

In [7]:
# Load the training data
with open('training_data.pkl', 'rb') as f:
    training_data = pickle.load(f)

# Separate features and labels
features = [data[0] for data in training_data]
labels = [data[1] for data in training_data]

# Initialize a model (here, a Random Forest classifier as an example)
model = RandomForestClassifier(n_estimators=100)

# Train the model
model.fit(features, labels)

# Save the model
with open('coin_classifier.pkl', 'wb') as f:
    pickle.dump(model, f)